In [2]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecode API Key: ")

In [3]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [5]:
INDEX_NAME = "thirukural"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
existing_indexes

['cop16', 'cop16-v1', 'cop-ifp']

In [6]:
import time

if INDEX_NAME not in existing_indexes:
    pc.create_index(
        name=INDEX_NAME,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    while not pc.describe_index(INDEX_NAME).status["ready"]:
        time.sleep(1)

index = pc.Index(INDEX_NAME)

In [71]:
import getpass
import os

# if not os.environ.get("OPENAI_API_KEY"):
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [72]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [83]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="../data/tamil_thirukkural_train.csv", 
                   encoding="utf-8",
                   content_columns=["explanation"],
                   metadata_columns=["Number", "kural", "adikaram_name",
                                     "iyal_name", "paul_name", "paul_translation",
                                     "mk", "mv", "sp"])

data = loader.load()

print(data)

[Document(metadata={'source': '../data/tamil_thirukkural_train.csv', 'row': 0, 'Number': '409', 'kural': 'மேற்பிறந்தா ராயினும் கல்லாதார் கீழ்ப்பிறந்தும் கற்றார் அனைத்திலர் பாடு.', 'adikaram_name': 'கல்லாமை', 'iyal_name': 'அரசியல்', 'paul_name': 'பொருட்பால்', 'paul_translation': 'Wealth', 'mk': 'கற்றவர் என்ற பெருமை, உயர்ந்தவர் தாழ்ந்தவர் என்ற வேறுபாட்டைப் போக்கிவிடும்', 'mv': 'கல்லாதவர் உயர்ந்த குடியில் பிறந்தவராக இருப்பினும் தாழ்ந்த குடியில் பிறந்திருந்தும் கல்விக் கற்றவரைப் போன்ற பெருமை இல்லாதவரே.', 'sp': 'படிக்காதவர் மேல்சாதியில் பிறந்திருந்தாலும், கீழ்சாதியில் பிறந்திருந்தும் படித்தவர் அளவிற்குப் பெருமை இல்லாதவரே.'}, page_content='explanation: The unlearned, though born in a high caste, are not equal in dignity to the learned; though they may have been born in a low caste'), Document(metadata={'source': '../data/tamil_thirukkural_train.csv', 'row': 1, 'Number': '415', 'kural': 'இழுக்கல் உடையுழி ஊற்றுக்கோல் அற்றே ஒழுக்க முடையார்வாய்ச் சொல்.', 'adikaram_name': 'கேள்வி', 'iyal_name': '

In [84]:
data[0].metadata

{'source': '../data/tamil_thirukkural_train.csv',
 'row': 0,
 'Number': '409',
 'kural': 'மேற்பிறந்தா ராயினும் கல்லாதார் கீழ்ப்பிறந்தும் கற்றார் அனைத்திலர் பாடு.',
 'adikaram_name': 'கல்லாமை',
 'iyal_name': 'அரசியல்',
 'paul_name': 'பொருட்பால்',
 'paul_translation': 'Wealth',
 'mk': 'கற்றவர் என்ற பெருமை, உயர்ந்தவர் தாழ்ந்தவர் என்ற வேறுபாட்டைப் போக்கிவிடும்',
 'mv': 'கல்லாதவர் உயர்ந்த குடியில் பிறந்தவராக இருப்பினும் தாழ்ந்த குடியில் பிறந்திருந்தும் கல்விக் கற்றவரைப் போன்ற பெருமை இல்லாதவரே.',
 'sp': 'படிக்காதவர் மேல்சாதியில் பிறந்திருந்தாலும், கீழ்சாதியில் பிறந்திருந்தும் படித்தவர் அளவிற்குப் பெருமை இல்லாதவரே.'}

In [85]:
print(data[0].page_content)

explanation: The unlearned, though born in a high caste, are not equal in dignity to the learned; though they may have been born in a low caste


In [86]:
for doc in data:
    doc.metadata["Muthuvel Karunanidhi"] = doc.metadata["mk"]
    del doc.metadata["mk"]
    doc.metadata["Solomon Pappaiah"] = doc.metadata["sp"]
    del doc.metadata["sp"]
    doc.metadata["Mu Varadarasanar"] = doc.metadata["mv"]
    del doc.metadata["mv"]
    doc.page_content = doc.page_content.removeprefix("explanation: ")
    

In [87]:
data[10].metadata

{'source': '../data/tamil_thirukkural_train.csv',
 'row': 10,
 'Number': '463',
 'kural': 'ஆக்கம் கருதி முதலிழக்கும் செய்வினை ஊக்கார் அறிவுடை யார்.',
 'adikaram_name': 'தெரிந்து செயல்வகை',
 'iyal_name': 'அரசியல்',
 'paul_name': 'பொருட்பால்',
 'paul_translation': 'Wealth',
 'Muthuvel Karunanidhi': 'பெரும் ஆதாயம் கிட்டுமென்று எதிர்பார்த்துக் கை முதலையும் இழந்து விடக்கூடிய காரியத்தை அறிவுடையவர்கள் செய்யமாட்டார்கள்',
 'Solomon Pappaiah': 'வரும் லாபத்தை எண்ணி, இருக்கும் முதலையும் இழந்துவிடுவதற்கு ஏற்ற செயலை அறிவுள்ளவர் செய்யமாட்டார்.',
 'Mu Varadarasanar': 'பின் விளையும் ஊதியத்தைக் கருதி இப்போது கையில் உள்ள முதலை இழந்து விடக் காரணமாச் செயலை அறிவுடையோர் மேற்க்கொள்ள மாட்டார்.'}

In [88]:
data[10].page_content

'Wise men will not, in the hopes of profit, undertake works that will consume their principal'

In [89]:
batches = [data[i:i + 100] for i in range(0, len(data), 100)]

In [90]:
len(batches)

11

In [91]:
len(data)

1064

In [92]:
for batch in batches:
    vector_store.add_documents(documents=batch)

In [103]:
results = vector_store.similarity_search_with_score(
    "Stealing from bank is crime",
    k=10,
)

from langchain_community.document_transformers import LongContextReorder

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(results)

for res, score in reordered_docs:
    print(f"{score} - *{res.page_content} [{res.metadata}]")

0.304768384 - *They cannot walk steadfastly, according to rule, who eagerly desire to defraud others [{'Mu Varadarasanar': 'களவு செய்து பிறர் பொருள் கொள்ளுதலில் மிக்க விருப்பம் உடையவர், அளவு (சிக்கனம்) போற்றி வாழும் நெறியில் நின்று ஒழுக மாட்டார்.', 'Muthuvel Karunanidhi': 'ஓர் எல்லைக்குட்பட்டு வாழ்வைச் செம்மையாக அமைத்துக் கொள்ளாதவர்கள், களவு செய்து பிறர் பொருளைக் கொள்வதில் நாட்டமுடையவராவார்கள்', 'Number': '286', 'Solomon Pappaiah': 'உயிர்களை நேசிக்கும் ஆசை இல்லாதவரே அடுத்தவர் பொருளைத் திருடும் பேராசை உடையவர் ஆவர்.', 'adikaram_name': 'கள்ளாமை', 'iyal_name': 'துறவறவியல்', 'kural': 'அளவின்கண் நின்றொழுகல் ஆற்றார் களவின்கண் கன்றிய காத லவர்.', 'paul_name': 'அறத்துப்பால்', 'paul_translation': 'Virtue', 'row': 86.0, 'source': '../data/tamil_thirukkural_test.csv'}]
0.292522609 - *Commit not evil, saying, "I am poor": if you do, you will become poorer still [{'Mu Varadarasanar': 'யான் வறியவன் என்று நினைத்துத் தீய செயல்களைச் செய்யக்கூடாது, செய்தால் மீண்டும் வறியவன் ஆகி வருந்துவான்.', 'Muthuvel Ka

In [104]:
reordered_docs

[(Document(id='631a0dfb-081e-494f-8101-8a5645965463', metadata={'Mu Varadarasanar': 'களவு செய்து பிறர் பொருள் கொள்ளுதலில் மிக்க விருப்பம் உடையவர், அளவு (சிக்கனம்) போற்றி வாழும் நெறியில் நின்று ஒழுக மாட்டார்.', 'Muthuvel Karunanidhi': 'ஓர் எல்லைக்குட்பட்டு வாழ்வைச் செம்மையாக அமைத்துக் கொள்ளாதவர்கள், களவு செய்து பிறர் பொருளைக் கொள்வதில் நாட்டமுடையவராவார்கள்', 'Number': '286', 'Solomon Pappaiah': 'உயிர்களை நேசிக்கும் ஆசை இல்லாதவரே அடுத்தவர் பொருளைத் திருடும் பேராசை உடையவர் ஆவர்.', 'adikaram_name': 'கள்ளாமை', 'iyal_name': 'துறவறவியல்', 'kural': 'அளவின்கண் நின்றொழுகல் ஆற்றார் களவின்கண் கன்றிய காத லவர்.', 'paul_name': 'அறத்துப்பால்', 'paul_translation': 'Virtue', 'row': 86.0, 'source': '../data/tamil_thirukkural_test.csv'}, page_content='They cannot walk steadfastly, according to rule, who eagerly desire to defraud others'),
  0.304768384),
 (Document(id='e7a16cb2-ec2b-4d01-9d57-70e1a8fc9d25', metadata={'Mu Varadarasanar': 'யான் வறியவன் என்று நினைத்துத் தீய செயல்களைச் செய்யக்கூடாது, செய்தால்

In [116]:
context_list = []

for doc in reordered_docs:
    context_list.append((f"Number: {doc[0].metadata["Number"]}\n"
               f"Thirukural: {doc[0].metadata["kural"]}\n"
               f"Short Explanation: {doc[0].page_content}\n"
               f"Adhikaram: {doc[0].metadata["adikaram_name"]}\n"
               f"Iyal: {doc[0].metadata["iyal_name"]}\n"
               f"Paal: {doc[0].metadata["paul_name"]}\n"
               f"Paal in English: {doc[0].metadata["paul_translation"]}\n"
               f"Explanation by Mu Varadarasanar: {doc[0].metadata["Mu Varadarasanar"]}\n"
               f"Explanation by Solomon Pappaiah {doc[0].metadata["Solomon Pappaiah"]}\n"
               f"Explanation by Muthuvel Karunanidhi: {doc[0].metadata["Muthuvel Karunanidhi"]}\n"))


In [118]:
context_str = "\n---\n".join(context_list)
print(context_str)

Number: 286
Thirukural: அளவின்கண் நின்றொழுகல் ஆற்றார் களவின்கண் கன்றிய காத லவர்.
Short Explanation: They cannot walk steadfastly, according to rule, who eagerly desire to defraud others
Adhikaram: கள்ளாமை
Iyal: துறவறவியல்
Paal: அறத்துப்பால்
Paal in English: Virtue
Explanation by Mu Varadarasanar: களவு செய்து பிறர் பொருள் கொள்ளுதலில் மிக்க விருப்பம் உடையவர், அளவு (சிக்கனம்) போற்றி வாழும் நெறியில் நின்று ஒழுக மாட்டார்.
Explanation by Solomon Pappaiah உயிர்களை நேசிக்கும் ஆசை இல்லாதவரே அடுத்தவர் பொருளைத் திருடும் பேராசை உடையவர் ஆவர்.
Explanation by Muthuvel Karunanidhi: ஓர் எல்லைக்குட்பட்டு வாழ்வைச் செம்மையாக அமைத்துக் கொள்ளாதவர்கள், களவு செய்து பிறர் பொருளைக் கொள்வதில் நாட்டமுடையவராவார்கள்

---
Number: 205
Thirukural: இலன் என்று தீயவை செய்யற்க செய்யின் இலனாகும் மற்றும் பெயர்த்து.
Short Explanation: Commit not evil, saying, "I am poor": if you do, you will become poorer still
Adhikaram: தீவினையச்சம்
Iyal: இல்லறவியல்
Paal: அறத்துப்பால்
Paal in English: Virtue
Explanation by Mu Varadarasanar

In [110]:
reordered_docs[0][0].metadata

{'Mu Varadarasanar': 'களவு செய்து பிறர் பொருள் கொள்ளுதலில் மிக்க விருப்பம் உடையவர், அளவு (சிக்கனம்) போற்றி வாழும் நெறியில் நின்று ஒழுக மாட்டார்.',
 'Muthuvel Karunanidhi': 'ஓர் எல்லைக்குட்பட்டு வாழ்வைச் செம்மையாக அமைத்துக் கொள்ளாதவர்கள், களவு செய்து பிறர் பொருளைக் கொள்வதில் நாட்டமுடையவராவார்கள்',
 'Number': '286',
 'Solomon Pappaiah': 'உயிர்களை நேசிக்கும் ஆசை இல்லாதவரே அடுத்தவர் பொருளைத் திருடும் பேராசை உடையவர் ஆவர்.',
 'adikaram_name': 'கள்ளாமை',
 'iyal_name': 'துறவறவியல்',
 'kural': 'அளவின்கண் நின்றொழுகல் ஆற்றார் களவின்கண் கன்றிய காத லவர்.',
 'paul_name': 'அறத்துப்பால்',
 'paul_translation': 'Virtue',
 'row': 86.0,
 'source': '../data/tamil_thirukkural_test.csv'}

In [111]:
reordered_docs[0][0].page_content

'They cannot walk steadfastly, according to rule, who eagerly desire to defraud others'